In [2]:
#importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, StratifiedKFold
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from skforecast.Sarimax import Sarimax
import requests
%matplotlib inline

In [3]:
#FRED api key: e702d160a3e348cf149d4a5c3faf0d0c

In [14]:
oil_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=DCOILBRENTEU&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json'
oil_req = requests.get(oil_url)
oil_json = oil_req.json()

In [15]:
oil_json['observations']

[{'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-20',
  'value': '18.63'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-21',
  'value': '18.45'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-22',
  'value': '18.55'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-25',
  'value': '18.60'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-26',
  'value': '18.63'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-27',
  'value': '18.60'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-28',
  'value': '18.60'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-05-29',
  'value': '18.58'},
 {'realtime_start': '2024-03-04',
  'realtime_end': '2024-03-04',
  'date': '1987-06-01',
  'value': '18.65'},
 

In [16]:
oil_df = pd.json_normalize(oil_json['observations'])
oil_df.head()

,realtime_start,realtime_end,date,value
0,2024-03-04,2024-03-04,1987-05-20,18.63
1,2024-03-04,2024-03-04,1987-05-21,18.45
2,2024-03-04,2024-03-04,1987-05-22,18.55
3,2024-03-04,2024-03-04,1987-05-25,18.60
4,2024-03-04,2024-03-04,1987-05-26,18.63


In [86]:
series_list = ['DCOILBRENTEU', 'DPCCRV1Q225SBEA', 'DSPIC96', 'EXPGS', 'GDP', 'IMPGS', 'INDPRO', 'WM2NS', 'M2V', 'NASDAQCOM', 'CORESTICKM159SFRBATL', 'DEXUSUK', 'RECPROUSM156N', 'UNRATE', 'DEXJPUS', 'T10Y2Y']
def make_dataframe(series_list):
    base_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=DFEDTAR&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json&observation_start=1987-08-11'
    base_request = requests.get(base_url)
    base_json = json = base_request.json()
    base_df = pd.json_normalize(base_json['observations'])
    baseu_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=DFEDTARU&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json&observation_start=1987-08-11'
    baseu_request = requests.get(baseu_url)
    baseu_json = baseu_request.json()
    baseu_df = pd.json_normalize(baseu_json['observations'])
    merge_df = pd.concat([base_df, baseu_df], ignore_index=True)
    rgdp_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=GDPC1&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json&observation_start=1987-08-11'
    rgdp_request = requests.get(rgdp_url)
    rgdp_json = rgdp_request.json()
    rgdp_df = pd.json_normalize(rgdp_json['observations'])
    rgdp_df['rgdp'] = rgdp_df['value']
    rgdp_df = rgdp_df.drop(columns = 'value')
    gdppot_url = 'https://api.stlouisfed.org/fred/series/observations?series_id=GDPPOT&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json&observation_start=1987-08-11'
    gdppot_request = requests.get(gdppot_url)
    gdppot_json = gdppot_request.json()
    gdppot_df = pd.json_normalize(gdppot_json['observations'])
    gdppot_df['gdppot'] = gdppot_df['value']
    gdppot_df = gdppot_df.drop(columns = 'value')
    merge_df = pd.merge_ordered(merge_df, rgdp_df, fill_method="none", left_by="date")
    merge_df = pd.merge_ordered(merge_df, gdppot_df, fill_method="none", left_by="date")
    merge_df['date'] = pd.to_datetime(merge_df['date'])
    merge_df['rgdp'] = merge_df['rgdp'].replace('.', np.nan)
    merge_df['gdppot'] = merge_df['gdppot'].replace('.', np.nan)
    merge_df['rgdp'] = merge_df['rgdp'].astype(float)
    merge_df['gdppot'] = merge_df['gdppot'].astype(float)
    merge_df['gdp_gap'] = 100*((merge_df['rgdp'] - merge_df['gdppot'])/merge_df['rgdp'])
    merge_df = merge_df.drop(columns = ['realtime_start', 'realtime_end', 'gdppot', 'rgdp'])
    for series in series_list:
        url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key=e702d160a3e348cf149d4a5c3faf0d0c&file_type=json&observation_start=1987-08-11'
        req = requests.get(url)
        json = req.json()
        df = pd.json_normalize(json['observations'])
        df = df.drop(columns = ['realtime_start', 'realtime_end'])
        df['date'] = pd.to_datetime(df['date'])
        df[series] = df['value']
        df = df.drop(columns = 'value')
        merge_df = pd.merge_ordered(merge_df, df, fill_method="none", left_by="date")
    merge_df = merge_df.set_index('date')
    return merge_df

In [88]:
merge_df = make_dataframe(series_list = series_list)
merge_df.head()

,value,gdp_gap,DCOILBRENTEU,DPCCRV1Q225SBEA,DSPIC96,EXPGS,GDP,IMPGS,INDPRO,WM2NS,M2V,NASDAQCOM,CORESTICKM159SFRBATL,DEXUSUK,RECPROUSM156N,UNRATE,DEXJPUS,T10Y2Y
date,,,,,,,,,,,,,,,,,,
1987-08-11,6.625,NaN,19.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,449.36,NaN,1.5697,NaN,NaN,151.77,1.04
1987-08-12,6.625,NaN,19.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,449.23,NaN,1.5790,NaN,NaN,151.25,1.05
1987-08-13,6.625,NaN,19.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451.55,NaN,1.5797,NaN,NaN,151.45,0.99
1987-08-14,6.625,NaN,19.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451.61,NaN,1.5890,NaN,NaN,150.00,0.96
1987-08-15,6.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#notebook substance imported over to pipeline construction notebook, work continues there, but wanted a separate notebook for this 
#so that I didnt screw up my old data